In [1]:
import numpy as np

In [48]:
from datetime import datetime

def datetime_conversion(str):
    return datetime.strptime(str, '%d%b%Y')

def cal_dur(t_e, t_d):
    date1 = datetime_conversion(t_e)
    date2 = datetime_conversion(t_d)
    delta = date1 - date2
    #print(delta)
    return int(delta.days)
    
def price_xyz(A_t_d, B, t_e, r, sigma, t_d, numsteps):
    
    #Calculate time step
    duration = cal_dur(t_e, t_d)
    ts = duration/numsteps

    #Calculate probabilities of moving up and down, underlying log-normal
    num_pu =  np.exp(r*ts/2) - np.exp(-1*sigma*np.sqrt(ts/2))
    den_pu = np.exp(sigma*np.sqrt(ts/2)) - np.exp(-1*sigma*np.sqrt(ts/2))
    p_u = np.square(num_pu/den_pu)
    
    num_pd =  np.exp(1*sigma*np.sqrt(ts/2)) - np.exp(r*ts/2)
    den_pd = np.exp(sigma*np.sqrt(ts/2)) - np.exp(-1*sigma*np.sqrt(ts/2))
    p_d = np.square(num_pd/den_pd)
    
    #Upward and downward multiplier
    j_u = np.exp(sigma*np.sqrt(ts))
    j_d = np.exp(-1*sigma*np.sqrt(ts))
    
    #1 time step backward multiplier
    disc_1ts = np.exp(-1*r*ts)
    
    #Make a numpy tree for underlying asset: abc
    abc_price = np.zeros([2*numsteps+1, numsteps+1])
    abc_price[numsteps,0] = A_t_d
    
    for i in range( numsteps):
        for j in range(2*numsteps+1):
            if abc_price[j, i]!=0:
                abc_price[j+1, i+1] = abc_price[j, i] * j_u
                abc_price[j-1, i+1] = abc_price[j, i] * j_d
                abc_price[j, i+1] = abc_price[j, i]
            #abc_price[j, i] = A_t_d * (j_d**j)*(j_u**(i-j))
    
    #Make a numpy tree for xyz: option
    xyz_price = np.zeros([2*numsteps+1, numsteps+1])
    
    #Leaf nodes correction
    for j in range(2*numsteps+1):
        xyz_price[j, numsteps] = max(abc_price[j, numsteps] - B, 0)
        
    #Internal Node Correction
    for i in reversed(range(0, numsteps)):
        for j in range(2*numsteps+1):
            if abc_price[j, i]!=0:
                xyz_price[j, i] = (p_u*xyz_price[j+1, i+1] + (1-p_u-p_d)*xyz_price[j, i+1] + p_d*xyz_price[j-1, i+1])*disc_1ts

        #Un-comment this to print the whole xyz_pricing tree
#     for row in xyz_price:
#         for val in row:
#             print(val, end=" ")
#         print()

    return xyz_price[numsteps,0]
price_xyz(30, 5, "01Jul2021", 0.02, 0.13, "06Jan2021", 5)

35.2
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.7798639028859753 0.37827972759805284 0.0 
0.0 0.0 2.531641330258995 2.779196810137277 2.644548074184935 1.4149300658387904 
0.0 5.016929993129532 6.152605074762977 7.366377576931926 8.441422851740711 8.872559315972083 
8.895721550493692 11.195617431043008 14.005285875788143 17.346792108940683 21.129921015416617 25.00000000000001 
0.0 24.558710434348995 30.990156871841297 38.935152196260574 48.56933837454895 59.87627693642595 
0.0 0.0 67.72084676499155 85.62089978386886 107.90824636005705 135.29771030439443 
0.0 0.0 0.0 186.58081609979405 236.23116061248766 298.3997702418117 
0.0 0.0 0.0 0.0 513.7349246901518 651.1149172218592 
0.0 0.0 0.0 0.0 0.0 1413.8764357268503 


8.895721550493692

In [53]:
def volrisk_xyz(A_t_d, B, t_e, r, sigma, t_d, numsteps):
    #Based on diffrentiation using the first principle
    delta = 0.000000001
    v1 = price_xyz(A_t_d, B, t_e, r, sigma, t_d, numsteps)
    v2 = price_xyz(A_t_d, B, t_e, r, sigma+delta, t_d, numsteps)
    return (v2-v1)/(delta)
print(volrisk_xyz(30, 5, "01Jul2021", 0.02, 0.13, "06Jan2021", 5))

35.2
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.7798639028859753 0.37827972759805284 0.0 
0.0 0.0 2.531641330258995 2.779196810137277 2.644548074184935 1.4149300658387904 
0.0 5.016929993129532 6.152605074762977 7.366377576931926 8.441422851740711 8.872559315972083 
8.895721550493692 11.195617431043008 14.005285875788143 17.346792108940683 21.129921015416617 25.00000000000001 
0.0 24.558710434348995 30.990156871841297 38.935152196260574 48.56933837454895 59.87627693642595 
0.0 0.0 67.72084676499155 85.62089978386886 107.90824636005705 135.29771030439443 
0.0 0.0 0.0 186.58081609979405 236.23116061248766 298.3997702418117 
0.0 0.0 0.0 0.0 513.7349246901518 651.1149172218592 
0.0 0.0 0.0 0.0 0.0 1413.8764357268503 
35.2
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.0 0.0 0.0 
0.0 0.0 0.0 0.7798638748974951 0.3782797030463189 0.0 
0.0 0.0 2.5316412859178126 2.779196758154956 2.644548013864644 1.4149299897197603 
0.0 5.016929942978163 6.152605017009324 7.36637751109654 8.4414